<a href="https://colab.research.google.com/github/denisangelo/Federated_Learning/blob/main/Notebooks/Centralized_CIFAR_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Instalando a plataforma/bibliotecas e suas dependências
!pip install -q flwr[simulation]
!pip install torch
!pip install torchvision
!pip install matplotlib

# Importando as bibliotecas
import flwr as fl
import warnings
from collections import OrderedDict
import sys
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from tqdm import tqdm

# Definindo o dispositivo de execução
warnings.filterwarnings("ignore", category=UserWarning)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE} using Pytorch{torch.__version__} and Flower{fl.__version__}")
# Definindo a rede neural
class Cifar10CnnModel(nn.Module):
 def __init__(self) -> None:
        super(Cifar10CnnModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
 def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)
# Salvando o melhor modelo
def saveModel(net):
     path = "./bestModelnnCifar10.pth"
     torch.save(net.state_dict(), path)
# Avaliando a acurácia no conjunto de dados de testes
def testAccuracy(net,testloader):
    net.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data[0].to(DEVICE), data[1].to(DEVICE)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct * 100 / total
    return accuracy
# Avaliando o erro no conjunto de dados de testes
def testLoss(net, testloader):
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, lossT = 0, 0, 0.0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data[0].to(DEVICE), data[1].to(DEVICE)
            outputs = net(images)
            lossT += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
    correct += (predicted == labels).sum().item()
    return lossT

    # Sumarizando os resultados de erro e acurácia do modelo
def summarize_results (epoch_list,accuracy_list,accuracyT_list,loss_list,loss_listT):
    # Plotanto o gráfico de acurácia de treinamento e teste
    plt.subplot (211)
    plt.plot(epoch_list,accuracy_list)
    plt.plot(epoch_list,accuracyT_list)
    plt.legend(['Acurácia Treino', 'Acurácia Teste'])
    plt.xlabel('Época')
    plt.ylabel('Acurácia')
    plt.grid(True)
    plt.title('Acurácia vs. No. de épocas')
    plt.tight_layout()
    # Plotanto o gráfico de erro de treinamento e teste
    plt.subplot (212)
    plt.plot(epoch_list,loss_list)
    plt.plot(epoch_list,loss_listT)
    # Encontrando a posição de menor perda de validação
    plt.legend(['Erro Treino', 'Erro Teste'])
    plt.xlabel('Época')
    plt.ylabel('Erro')
    plt.grid(True)
    plt.title('Erro vs. No. de épocas')
    plt.tight_layout()
    # Salvando o plot em arquivo
    filename = sys.argv [0].split ('/') [-1]
    plt.savefig (filename + '_plot.png')
    plt.close ()
# Realizando o Treinamento do modelo na rede neural
def train(net, trainloader, epochs):
    print(f"Treinando {epochs} época(s) com {len(trainloader)} lotes cada")
    accuracy_list=[]
    epoch_list=[]
    loss_list= []
    accuracyT_list=[]
    loss_listT=[]
    best_accuracy = 0.0
    # Definindo uma função de perda e um otimizador
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    loss_listT=[]
    best_accuracy = 0.0
    # Definindo uma função de perda e um otimizador
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    for num_epoch in range(epochs):
        correct, total, epochs_loss = 0, 0, 0.0
        for i,(images, labels) in enumerate(tqdm(trainloader,0)):
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
    # Coletando as métricas de erro e acurácia
            epochs_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epochs_loss = epochs_loss / (len(trainloader)*batch_size)
        epochs_loss = epochs_loss.detach().cpu().numpy()
        epochs_acc = correct * 100 / total
        accuracy_list.append(epochs_acc)
        epoch_list.append(num_epoch+1)
        loss_list.append(epochs_loss)
        print("Época {:03d}, Erro de Treino: {:.6f}, Acurácia: {:.2f}%".format(num_epoch+1, epochs_loss, epochs_acc))
        print("Avaliando a performance do modelo ao conjunto de teste")
        epochs_acc = testAccuracy(net,testloader)
        if epochs_acc > best_accuracy:
          print("Salvando modelo com melhor acurácia no conjunto de teste")
          saveModel(net)
          best_accuracy = epochs_acc
          accuracyT_list.append(best_accuracy)
          print("Para a época", num_epoch+1,'a acurácia média do modelo aplicada ao conjunto de teste é: % .2f %%' % (best_accuracy))
          print("Avaliando o erro médio do modelo no conjunto de teste")
        for i,(images, labels) in enumerate(testloader,0): 
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            lossT = criterion(net(images), labels)
            lossT.backward()
            optimizer.step()
        epochs_lossT = testLoss(net,testloader)
        epochs_lossT += lossT
        epochs_lossT= epochs_lossT / (len(testloader)*batch_size)
        epochs_lossT = epochs_lossT.detach().cpu().numpy()
        loss_listT.append(epochs_lossT)
        print("Para a época", num_epoch+1,'o erro médio do modelo aplicada ao conjunto de teste é: %.6f' % (epochs_lossT))
        # Sumarizando os resultados
        summarize_results(epoch_list,accuracy_list,accuracyT_list,loss_list,loss_listT)
    return accuracy_list, epoch_list, best_accuracy,loss_list, accuracyT_list,loss_listT

    # Carregando e Normalizando os Dados CIFAR10
    # Definindo as Transformações a serem aplicadas às imagens
def load_data():
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize( (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    trainset = CIFAR10(root="./data", train=True,
    download=True, transform=transform)
    testset = CIFAR10(root="./data", train=False,
    download=True, transform=transform)
    trainloader = DataLoader(trainset, batch_size=batch_size,shuffle=True,num_workers=0)
    print("O número de Imagens no conjunto de treinamento é: ", len(trainloader)*batch_size)
    testloader = DataLoader(testset,batch_size=batch_size,shuffle=False,num_workers=0)
    num_examples = {"trainset": len(trainset)*batch_size, "testset": len(testset)*batch_size}
    print("O número de Imagens no conjunto de teste é: ", len(testloader)*batch_size)
    print("O número de Lotes de Treinamento por Épocas é:", len(trainloader))
    print("O número de Lotes de Teste por Épocas é:", len(testloader))
    return trainloader, testloader, num_examples

 # Programa Principal
if __name__ == "__main__":
 # Estabelecendo os parâmetros da rede
 batch_size = 100
 epochs =  10
 # Carregando o modelo e os dados
 net = Cifar10CnnModel().to(DEVICE)
 trainloader, testloader, num_examples = load_data()
 # Iniciando o treinamento e teste
 print("O modelo será executado em", DEVICE, "device")
 print("Iniciando o Treinamento")
 start_time = time.time()
 train(net, trainloader=trainloader, epochs=epochs)
 end_time = time.time()
 # Finalizando o treinamento e teste
 print("Finalizando o Treinamento")
 time_taken = end_time - start_time
 print('Tempo Total de Treinamento em minutos:% .2f' % (time_taken/60))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Training on cpu using Pytorch2.0.1+cu118 and Flower1.4.0
Files already downloaded and verified
Files already downloaded and verified
O número de Imagens no conjunto de treinamento é:  50000
O número de Imagens no conjunto de teste é:  10000
O número de Lotes de Treinamento por Épocas é: 500
O número de Lotes de Teste por Épocas é: 100
O modelo será executado em cpu device
Iniciando o Treinamento
Treinando 10 época(s) com 500 lotes cada


100%|██████████| 500/500 [00:40<00:00, 12.46it/s]


Época 001, Erro de Treino: 0.016919, Acurácia: 38.05%
Avaliando a performance do modelo ao conjunto de teste


100%|██████████| 100/100 [00:04<00:00, 21.32it/s]


Salvando modelo com melhor acurácia no conjunto de teste
Para a época 1 a acurácia média do modelo aplicada ao conjunto de teste é:  45.74 %
Avaliando o erro médio do modelo no conjunto de teste


...

Época 010, Erro de Treino: 0.009435, Acurácia: 66.64%
Avaliando a performance do modelo ao conjunto de teste


100%|██████████| 100/100 [00:05<00:00, 19.53it/s]


Salvando modelo com melhor acurácia no conjunto de teste
Para a época 10 a acurácia média do modelo aplicada ao conjunto de teste é:  67.63 %
Avaliando o erro médio do modelo no conjunto de teste


100%|██████████| 100/100 [00:04<00:00, 21.86it/s]


Para a época 10 o erro médio do modelo aplicada ao conjunto de teste é: 0.008787
Finalizando o Treinamento
Tempo Total de Treinamento em minutos: 9.60
